In [18]:
import pandas as pd
pd.set_option("display.max_columns",None)

In [19]:
df = pd.read_csv("../tmp/aml_data.csv")
df.head()

,Time,Date,Sender_account,Receiver_account,Amount,Payment_currency,Received_currency,Sender_bank_location,Receiver_bank_location,Payment_type,Is_laundering,Laundering_type
0,10:35:19,2022-10-07,8724731955,2769355426,1459.15,UK pounds,UK pounds,UK,UK,Cash Deposit,0,Normal_Cash_Deposits
1,10:35:20,2022-10-07,1491989064,8401255335,6019.64,UK pounds,Dirham,UK,UAE,Cross-border,0,Normal_Fan_Out
2,10:35:20,2022-10-07,287305149,4404767002,14328.44,UK pounds,UK pounds,UK,UK,Cheque,0,Normal_Small_Fan_Out
3,10:35:21,2022-10-07,5376652437,9600420220,11895.00,UK pounds,UK pounds,UK,UK,ACH,0,Normal_Fan_In
4,10:35:21,2022-10-07,9614186178,3803336972,115.25,UK pounds,UK pounds,UK,UK,Cash Deposit,0,Normal_Cash_Deposits


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9504852 entries, 0 to 9504851
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Time                    object 
 1   Date                    object 
 2   Sender_account          int64  
 3   Receiver_account        int64  
 4   Amount                  float64
 5   Payment_currency        object 
 6   Received_currency       object 
 7   Sender_bank_location    object 
 8   Receiver_bank_location  object 
 9   Payment_type            object 
 10  Is_laundering           int64  
 11  Laundering_type         object 
dtypes: float64(1), int64(3), object(8)
memory usage: 870.2+ MB


In [5]:
df.shape

(9504852, 12)

In [7]:
df['Date']

0          2022-10-07
1          2022-10-07
2          2022-10-07
3          2022-10-07
4          2022-10-07
              ...    
9504847    2023-08-23
9504848    2023-08-23
9504849    2023-08-23
9504850    2023-08-23
9504851    2023-08-23
Name: Date, Length: 9504852, dtype: object

In [8]:
df['Time']

0          10:35:19
1          10:35:20
2          10:35:20
3          10:35:21
4          10:35:21
             ...   
9504847    10:57:01
9504848    10:57:06
9504849    10:57:06
9504850    10:57:11
9504851    10:57:12
Name: Time, Length: 9504852, dtype: object

In [20]:
df['Transaction_datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [21]:
df.drop(['Date', 'Time'], axis=1, inplace=True)

df['Sender_account'] = df['Sender_account'].astype(str)  # Convert BIGINT to str
df['Receiver_account'] = df['Receiver_account'].astype(str)  # Convert BIGINT to str
df['Amount'] = df['Amount'].astype(float)  # Ensure FLOAT type
df['Payment_currency'] = df['Payment_currency'].astype(str)
df['Received_currency'] = df['Received_currency'].astype(str)
df['Sender_bank_location'] = df['Sender_bank_location'].astype(str)
df['Receiver_bank_location'] = df['Receiver_bank_location'].astype(str)
df['Payment_type'] = df['Payment_type'].astype(str)
df['Is_laundering'] = df['Is_laundering'].astype(int)  # Ensure INT type
df['Laundering_type'] = df['Laundering_type'].astype(str)

In [ ]:
max_lengths = df.apply(lambda x: x.str.len().max() if x.dtype == "object" else None)
print(max_lengths)

df['Laundering_type'].head()

Sender_account            10.0
Receiver_account          10.0
Amount                     NaN
Payment_currency          15.0
Received_currency         15.0
Sender_bank_location      11.0
Receiver_bank_location    11.0
Payment_type              15.0
Is_laundering              NaN
Laundering_type           22.0
Transaction_datetime       NaN
dtype: float64


In [23]:
df['Payment_currency'].value_counts()

Payment_currency
UK pounds          9099293
Euro                117164
Turkish lira         27996
Swiss franc          27492
Dirham               27263
Pakistani rupee      27196
Naira                27143
US dollar            26061
Yen                  25562
Moroccan dirham      25395
Mexican Peso         24852
Albanian lek         24778
Indian rupee         24657
Name: count, dtype: int64

In [ ]:
import pyodbc
import pandas as pd

SQL_SERVER = r"DESKTOP-JSV1UOD\USER_ROOT"
SQL_DATABASE = "master"  
TABLE_NAME = 'Transactions'
CSV_FILE_PATH = "../tmp/aml_data.csv"


conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SQL_SERVER};Trusted_Connection=yes;Encrypt=Mandatory;TrustServerCertificate=Yes;"
try:

    conn = pyodbc.connect(conn_str)
    conn.autocommit = True  
    cursor = conn.cursor()

    DATABASE_NAME = "AMLDb"
    cursor.execute(f"SELECT name FROM sys.databases WHERE name = '{DATABASE_NAME}'")
    existing_db = cursor.fetchone()

    if existing_db:
        print(f"Database '{DATABASE_NAME}' already exists. No action needed.")
    else:

        cursor.execute(f"CREATE DATABASE {DATABASE_NAME}")
        print(f"Database '{DATABASE_NAME}' created successfully!")

    cursor.close()
    conn.close()

    conn_str = f"DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SQL_SERVER};DATABASE={DATABASE_NAME};Trusted_Connection=yes;Encrypt=Mandatory;TrustServerCertificate=Yes;"
    conn = pyodbc.connect(conn_str)
    conn.autocommit = True
    cursor = conn.cursor()

    create_table_query = f"""
        IF OBJECT_ID('{TABLE_NAME}', 'U') IS NULL
        CREATE TABLE {TABLE_NAME} (
            Time VARCHAR(255),
            Date VARCHAR(255),
            Sender_account BIGINT,
            Receiver_account BIGINT,
            Amount DECIMAL(18, 2),
            Payment_currency VARCHAR(500),
            Received_currency VARCHAR(255),
            Sender_bank_location VARCHAR(255),
            Receiver_bank_location VARCHAR(255),
            Payment_type VARCHAR(255),
            Is_laundering INT,
            Laundering_type VARCHAR(255)
        );
    """
    cursor.execute(create_table_query)
    conn.commit()

    print(f"Table '{TABLE_NAME}' is ready.")

    df = pd.read_csv(CSV_FILE_PATH)

    df["Sender_account"] = df["Sender_account"].astype(str)  
    df["Receiver_account"] = df["Receiver_account"].astype(str)
    df["Amount"] = df["Amount"].astype(float)
    df["Is_laundering"] = df["Is_laundering"].astype(int)
    print("conversion is completed..")

    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    print("spaces is removed..")

    insert_query = f"""
        INSERT INTO {TABLE_NAME} (Time, Date, Sender_account, Receiver_account, Amount, Payment_currency, Received_currency, Sender_bank_location, Receiver_bank_location, Payment_type, Is_laundering, Laundering_type)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """
    print("insert query..")

    cursor.fast_executemany = True

    batch_size = 5000  
    values = [tuple(row) for row in df.itertuples(index=False, name=None)]

    for i in range(0, len(values), batch_size):
        batch = values[i : i + batch_size]
        cursor.executemany(insert_query, batch)
        conn.commit()
        print(f"Inserted {len(batch)} rows...")

    print(f"Successfully inserted {len(df)} rows into '{TABLE_NAME}'.")

    cursor.close()
    conn.close()

except pyodbc.Error as e:
    print("Database Error:", e)

Database 'AMLDb' created successfully!
Table 'Transactions' is ready.
conversion is completed..
spaces is removed..
insert query..
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 5000 rows...
Inserted 50